In [36]:
import os
import nbformat as nbf
from nbformat.v4 import new_code_cell, new_markdown_cell

def create_ipynb_from_repo(repo_path, output_filename):
    nb = nbf.v4.new_notebook()
    cells = []

    for root, dirs, files in os.walk(repo_path):
        for file in files:
            if file.endswith('.md') or file.endswith('.py'):
                file_path = os.path.join(root, file)
                rel_path = os.path.relpath(file_path, repo_path)
                file_url = f"https://github.com/your_username/your_repo_name/blob/main/{rel_path}"

                cells.append(new_markdown_cell(f"## [{rel_path}]({file_url})"))
                
                with open(file_path, 'r') as f:
                    file_content = f.read()
                    if file.endswith('.md'):
                        cells.append(new_markdown_cell(file_content))
                    else:
                        cells.append(new_code_cell(file_content))

    nb['cells'] = cells

    with open(output_filename, 'w') as f:
        nbf.write(nb, f)

if __name__ == '__main__':
    repo_path = "."
    output_filename = "combined.ipynb"
    create_ipynb_from_repo(repo_path, output_filename)


In [37]:
import os
import yaml

def create_yaml_from_repo(repo_path, output_filename):
    def add_to_dict_hierarchy(d, path_parts, file):
        if len(path_parts) == 1:
            d[path_parts[0]] = file
        else:
            if path_parts[0] not in d:
                d[path_parts[0]] = {}
            add_to_dict_hierarchy(d[path_parts[0]], path_parts[1:], file)

    repo_dict = {}
    
    for root, dirs, files in os.walk(repo_path):
        # Ignore .git directories
        if ".git" in root:
            continue
        
        for file in files:
            file_path = os.path.join(root, file)
            rel_path = os.path.relpath(file_path, repo_path)
            path_parts = rel_path.split(os.sep)

            add_to_dict_hierarchy(repo_dict, path_parts, file)

    with open(output_filename, 'w') as f:
        yaml.dump(repo_dict, f, sort_keys=True, indent=4)

if __name__ == '__main__':
    repo_path = "."
    output_filename = "repository_structure.yaml"
    create_yaml_from_repo(repo_path, output_filename)


In [40]:
%pip install nbconvert
import nbconvert
import os

def convert_ipynb_to_pdf(input_file, output_file):
    os.system(f"jupyter nbconvert --to pdf {input_file} --output {output_file}")

if __name__ == "__main__":
    input_file = "combined.ipynb"
    output_file = "Digital_Twin_Guide.pdf"
    convert_ipynb_to_pdf(input_file, output_file)


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: C:\Users\todd_\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
